In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [29]:
df = spark.read.parquet("data/final/merge_jugadores.parquet")
df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Born: float (nullable = true)
 |-- MP: float (nullable = true)
 |-- Starts: float (nullable = true)
 |-- Min: float (nullable = true)
 |-- Gls: double (nullable = true)
 |-- Ast: double (nullable = true)
 |-- YellowC: float (nullable = true)
 |-- RedC: float (nullable = true)
 |-- PrgC: double (nullable = true)
 |-- PrgP: double (nullable = true)
 |-- PrgR: double (nullable = true)
 |-- Fls: double (nullable = true)
 |-- Off: double (nullable = true)
 |-- Crosses: double (nullable = true)
 |-- Recov: double (nullable = true)
 |-- Aerialwon%: float (nullable = true)
 |-- Tack_Def_3rd: double (nullable = true)
 |-- Tack_Mid_3rd: double (nullable = true)
 |-- Tack_Att_3rd: double (nullable = true)
 |-- Tkl%: float (nullable = true)
 |-- Tkl: double (nullable = true)
 |-- TklW: double (nullable = true)
 |-- Int: double (nullab

In [27]:
from pyspark.sql.functions import col

df = df.toPandas()

#df.to_csv("data/performnace3.csv", index=False)

In [31]:
# Ordenar por adjusted_score descendente
df_top = df.orderBy(col("adjusted_score").desc())

# Seleccionar columnas clave para visualizar
df_top.select(
    "Player", "Squad", "Pos",
    "adjusted_score", "performance_score", "penalty_score", "Value"
).show(50, truncate=False)  # Puedes ajustar el número de filas

+------------------+---------------+----------------+------------------+-----------------+--------------------+---------+
|Player            |Squad          |Pos             |adjusted_score    |performance_score|penalty_score       |Value    |
+------------------+---------------+----------------+------------------+-----------------+--------------------+---------+
|kevin paredes     |Wolfsburg      |Left Midfield   |22.398752145606597|22.898214        |0.49946219460336405 |5000000.0|
|juan cuadrado     |Atalanta       |Right Midfield  |21.11975944112744 |21.572994        |0.4532347910502959  |1000000.0|
|ademola lookman   |Atalanta       |Second Striker  |20.65285145200088 |20.963503        |0.3106514319102507  |4.0E7    |
|paulo dybala      |Roma           |Second Striker  |20.6037759806492  |20.841646        |0.23787021380880793 |2.0E7    |
|nicola zalewski   |Inter          |Left Midfield   |20.024474804636696|21.233036        |1.2085612366230678  |1.2E7    |
|thomas muller     |Baye

In [36]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# Definir una ventana por Posición, ordenada por adjusted_score descendente
window_spec = Window.partitionBy("Pos").orderBy(col("adjusted_score").desc())
df_2024_2025 = df.filter(col("Season") == "2024-2025")
# Aplicar ranking
df_ranked = df.withColumn("rank", rank().over(window_spec))

# Filtrar los mejores (rank = 1) por posición
df_top_by_position = df_ranked.filter(col("rank") == 1)

# Mostrar resultados
df_top_by_position.select(
    "Pos", "Player", "Squad", "Season", "adjusted_score", "performance_score", "penalty_score", "Value"
).orderBy("Pos").show(truncate=False)


+------------------+-----------------+---------------+---------+------------------+-----------------+--------------------+---------+
|Pos               |Player           |Squad          |Season   |adjusted_score    |performance_score|penalty_score       |Value    |
+------------------+-----------------+---------------+---------+------------------+-----------------+--------------------+---------+
|Attacking Midfield|florian wirtz    |Leverkusen     |2023-2024|12.950615006391056|13.1680975       |0.2174824896416597  |1.3E8    |
|Central Midfield  |jakub moder      |Brighton       |2024-2025|15.372779846191406|15.37278         |0.0                 |1.2E7    |
|Centre-Back       |giulian biancone |Nott'ham Forest|2022-2023|11.403867721557617|11.403868        |0.0                 |6000000.0|
|Centre-Forward    |antoine griezmann|Atlético Madrid|2022-2023|9.603654310232534 |9.705955         |0.10230024146424449 |2.5E7    |
|Defensive Midfield|joshua kimmich   |Bayern Munich  |2024-2025|11.04

In [ ]:
df.filter(col("Value")==)
df_sin_valor= df.select("Player", "Value")
df_sin_valor.show()

+--------------------+---------+
|              Player|    Value|
+--------------------+---------+
|      kevin akpoguma|2500000.0|
|       noahkai banks|6000000.0|
|    maximilian bauer|2000000.0|
|         stefan bell| 800000.0|
|    jeanuel belocian|    1.5E7|
|  sebastiaan bornauw|6000000.0|
|el chadaille bits...|    1.8E7|
|         anrie chase|3000000.0|
|       arthur chaves|5000000.0|
|      danny da costa|1500000.0|
|      ameen aldakhil|7500000.0|
|           eric dier|8000000.0|
|      danilho doekhi|    1.3E7|
|         tim drexler|3000000.0|
|        adam dzwigaa| 600000.0|
|        marco friedl|    1.2E7|
|     benedikt gimber|3000000.0|
|     matthias ginter|7000000.0|
|  jeffrey gouweleeuw|1500000.0|
| andreas hancheolsen|6000000.0|
+--------------------+---------+
only showing top 20 rows


In [26]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType


float_columns = [
    'Born','MP','Starts','Min','Gls','Ast','YellowC','RedC','PrgC','PrgP','PrgR',
    'Fls','Off','Crosses','Recov','Aerialwon%','Tack_Def_3rd','Tack_Mid_3rd','Tack_Att_3rd','Tkl%',
    'Tkl','TklW','Int','Blocks','Block_Shots','Bolck_Pass','Clearences','Err',
    'Pass_cmp','Pass_cmp%','Pass_cmp_Short%','Pass_cmp_Medium%','Pass_cmp_Long%',
    'xAG','xA','A-xAG','Pass_cmp_Att_3rd','PPA','CrsPA',
    'Touch_Def_3rd','Touch_Mid_3rd','Touch_Att_3rd','Touch_Att_Pen',
    'drib_Att','drib_Succ%','PrgDist','Carries_Att_3rd','Carries_Att_Pen',
    'fail_To_Gain_Control','Loss_Control_Tackle','Sh','SoT','G/Sh',
    'xG','npxG','npxG/Sh','G-xG','Height','Value'
]

for col_name in float_columns:
    df = df.withColumn(col_name, col(col_name).cast(FloatType()))
    
from pyspark.sql.functions import isnan, when, count

df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in float_columns
]).show()


+----+---+------+---+---+---+-------+----+----+----+----+---+---+-------+-----+----------+------------+------------+------------+----+---+----+---+------+-----------+----------+----------+---+--------+---------+---------------+----------------+--------------+---+---+-----+----------------+---+-----+-------------+-------------+-------------+-------------+--------+----------+-------+---------------+---------------+--------------------+-------------------+---+---+----+---+----+-------+----+------+-----+
|Born| MP|Starts|Min|Gls|Ast|YellowC|RedC|PrgC|PrgP|PrgR|Fls|Off|Crosses|Recov|Aerialwon%|Tack_Def_3rd|Tack_Mid_3rd|Tack_Att_3rd|Tkl%|Tkl|TklW|Int|Blocks|Block_Shots|Bolck_Pass|Clearences|Err|Pass_cmp|Pass_cmp%|Pass_cmp_Short%|Pass_cmp_Medium%|Pass_cmp_Long%|xAG| xA|A-xAG|Pass_cmp_Att_3rd|PPA|CrsPA|Touch_Def_3rd|Touch_Mid_3rd|Touch_Att_3rd|Touch_Att_Pen|drib_Att|drib_Succ%|PrgDist|Carries_Att_3rd|Carries_Att_Pen|fail_To_Gain_Control|Loss_Control_Tackle| Sh|SoT|G/Sh| xG|npxG|npxG/Sh|G-xG|He

In [17]:
df_unidos = df.toPandas()

df_unidos.sort_values(by="Player").to_csv("data/unidos.csv", index=False)

Py4JJavaError: An error occurred while calling o152.collectToPython.
: org.apache.spark.SparkException: [FAILED_READ_FILE.FILE_NOT_EXIST] Encountered error while reading file file:///c:/Universidad/Master%20BDDE%20UCM/TFM/TFM/data/final/merge_jugadores.parquet/part-00000-079ac795-768d-4994-8457-5ddb12e67746-c000.snappy.parquet. File does not exist. It is possible the underlying files have been updated.
You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved. SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.fileNotExistError(QueryExecutionErrors.scala:831)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:140)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:142)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:695)
	at org.ap

In [5]:
from pyspark.sql.functions import col, when, lit, udf
from pyspark.sql.types import FloatType
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from functools import reduce

# UDF para sumar valores de un vector (normalizado)
@udf(FloatType())
def sum_vector(vec):
    return float(sum(vec))

def calcular_penalty_score(df):
    
    return df.fillna(0, subset=[
        'YellowC', 'RedC', 'Fls', 'Off', 'Err', 'Loss_Control_Tackle', 'fail_To_Gain_Control'
    ]).withColumn("penalty_score",
        col("YellowC") * 2.0 +
        col("RedC") * 5.0 +
        col("Fls") * 1.0 +
        col("Err") * 2.0 +
        col("Loss_Control_Tackle") * 1.0 +
        col("fail_To_Gain_Control") * 0.5 +
        when(
            col("Pos").rlike("Forward|Winger|Striker|Attacking Midfield|Second Striker"),
            col("Off") * 1.5
        ).otherwise(0)
    )

def calcular_performance_score(df, position_metrics):

    def procesar_posicion_x_temporada(df, pos, metrics):
        # Obtener todas las temporadas
        seasons = [row["Season"] for row in df.select("Season").distinct().collect()]
        df_pos_season_list = []

        for season in seasons:
            df_season = df.filter((col("Pos") == pos) & (col("Season") == season)).fillna(0, subset=metrics)

            #combina todas las métricas en un solo vector numérico.
            assembler = VectorAssembler(inputCols=metrics, outputCol="features_vec")
            df_season = assembler.transform(df_season)

            # transforma cada métrica del vector entre 0 y 1, escalando en base a los valores mínimos y máximos de esa posición.
            scaler = MinMaxScaler(inputCol="features_vec", outputCol="scaled_features")
            scaler_model = scaler.fit(df_season)
            df_season = scaler_model.transform(df_season)

            #suma las variables escaladas para calcular el performance_score
            df_season = df_season.withColumn("performance_score", sum_vector("scaled_features")) \
                                 .drop("features_vec", "scaled_features") \
                                 .withColumn("Evaluated_Position", lit(pos))

            df_pos_season_list.append(df_season)
        #une los dataframes por temporada ordenandolo por nombre
        return reduce(lambda df1, df2: df1.unionByName(df2), df_pos_season_list)

    # Procesar todas las posiciones
    scored_dfs = []
    for pos, metrics in position_metrics.items():
        if metrics:  # solo si hay métricas para esa posición
            scored_dfs.append(procesar_posicion_x_temporada(df, pos, metrics))

    return reduce(lambda df1, df2: df1.unionByName(df2), scored_dfs)


In [31]:
position_metrics = {
    'Goalkeeper': [],
    'Centre-Back': [
        'Tkl', 'Int', 'Blocks', 'Clearences', 'Aerialwon%',
        'Touch_Def_3rd', 'Tack_Def_3rd', 'Pass_cmp%', 'PrgC', 'PrgP',
        'Tkl%', 'Block_Shots', 'Pass_cmp_Short%', 'Pass_Short',
        'Pass_cmp_Medium%', 'Pass_Medium', 'Pass_cmp_Long%', 'Pass_Long',
        'PrgDist'
    ],
    'Left-Back': [
        'PrgC', 'PrgP', 'Crosses', 'Recov', 'Tack_Def_3rd', 'Tkl%', 'Tkl', 'Int',
        'Block_Shots', 'Bolck_Pass', 'Clearences', 'Pass_cmp', 'Pass_cmp%',
        'Pass_cmp_Short%', 'Pass_Short', 'Pass_cmp_Medium%', 'Pass_Medium', 'xA',
        'A-xAG', 'CrsPA', 'Touch_Def_3rd', 'Touch_Att_3rd', 'drib_Att', 'drib_Succ%',
        'PrgDist', 'Carries_Att_3rd', 'Touch_Live'
    ],
    'Right-Back': [
        'PrgC', 'PrgP', 'Crosses', 'Recov', 'Tack_Def_3rd', 'Tkl%', 'Tkl', 'Int',
        'Block_Shots', 'Bolck_Pass', 'Clearences', 'Pass_cmp', 'Pass_cmp%',
        'Pass_cmp_Short%', 'Pass_Short', 'Pass_cmp_Medium%', 'Pass_Medium', 'xA',
        'A-xAG', 'CrsPA', 'Touch_Def_3rd', 'Touch_Att_3rd', 'drib_Att', 'drib_Succ%',
        'PrgDist', 'Carries_Att_3rd', 'Touch_Live'
    ],
    'Defensive Midfield': [
        'Tkl', 'Int', 'Blocks', 'Clearences', 'Aerialwon%', 'Touch_Def_3rd',
        'Touch_Mid_3rd', 'Tack_Def_3rd', 'Pass_cmp', 'Pass_cmp%', 'PrgC', 'PrgP',
        'Tkl%', 'Block_Shots', 'Pass_cmp_Short%', 'Pass_cmp_Long%',
        'Pass_cmp_Medium%', 'PrgDist', 'Recov',
        'Tack_Mid_3rd', 'Bolck_Pass', 'Touch_Live'
    ],
    'Central Midfield': [
        'Ast', 'PrgC', 'PrgP', 'Recov', 'Aerialwon%', 'Tack_Def_3rd', 'Tack_Mid_3rd',
        'Tack_Att_3rd', 'Tkl%', 'Tkl', 'Int', 'Bolck_Pass', 'Pass_cmp',
        'Pass_cmp%', 'Pass_cmp_Short%', 'Pass_Short', 'Pass_cmp_Medium%', 'Pass_Medium',
        'Pass_Long', 'Pass_cmp_Long%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA',
        'Touch_Def_3rd', 'Touch_Mid_3rd', 'Touch_Att_3rd', 'Tckl_Drib', 'Tckl_Drib%',
        'PrgDist', 'Carries_Att_3rd', 'Touch_Live'
    ],
    'Attacking Midfield': [
        'Ast', 'PrgC', 'PrgP', 'PrgR', 'Tack_Att_3rd', 'Tack_Mid_3rd', 'Bolck_Pass',
        'Pass_cmp', 'Pass_cmp%', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium',
        'Pass_cmp_Medium%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA',
        'Touch_Mid_3rd', 'Touch_Att_3rd', 'Touch_Att_Pen', 'drib_Att', 'drib_Succ%',
        'Tckl_Drib', 'Tckl_Drib%', 'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen',
        'Sh', 'SoT', 'G/Sh', 'Touch_Live'
    ],
    'Right Midfield': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Crosses', 'Tack_Mid_3rd', 'Tack_Att_3rd',
        'Tkl%', 'Tkl', 'Bolck_Pass', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium',
        'Pass_cmp_Medium%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA',
        'Touch_Mid_3rd', 'Touch_Att_3rd', 'drib_Att', 'drib_Succ%', 'Tckl_Drib',
        'Tckl_Drib%', 'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen',
        'Sh', 'SoT', 'G/Sh'
    ],
    'Left Midfield': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Crosses', 'Tack_Mid_3rd', 'Tack_Att_3rd',
        'Tkl%', 'Tkl', 'Bolck_Pass', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium',
        'Pass_cmp_Medium%', 'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA',
        'Touch_Mid_3rd', 'Touch_Att_3rd', 'drib_Att', 'drib_Succ%', 'Tckl_Drib',
        'Tckl_Drib%', 'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen',
        'Sh', 'SoT', 'G/Sh'
    ],
    'Left Winger': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Crosses', 'Tack_Att_3rd', 'Tkl%', 'Tkl',
        'Bolck_Pass', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%',
        'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA', 'Touch_Att_3rd',
        'Touch_Att_Pen', 'drib_Att', 'drib_Succ%', 'Tckl_Drib', 'Tckl_Drib%',
        'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen',
        'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG'
    ],
    'Right Winger': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Crosses', 'Tack_Att_3rd', 'Tkl%', 'Tkl',
        'Bolck_Pass', 'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%',
        'xAG', 'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'PPA', 'CrsPA', 'Touch_Att_3rd',
        'Touch_Att_Pen', 'drib_Att', 'drib_Succ%', 'Tckl_Drib', 'Tckl_Drib%',
        'PrgDist', 'Carries_Att_3rd', 'Carries_Att_Pen', 
        'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG'
    ],
    'Second Striker': [
        'Gls', 'Ast', 'PrgC', 'PrgP', 'PrgR', 'Tack_Att_3rd', 'Bolck_Pass',
        'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%', 'xAG', 'xA', 'A-xAG',
        'Pass_cmp_Att_3rd', 'PPA', 'Touch_Mid_3rd', 'Touch_Att_3rd', 'Touch_Att_Pen',
        'drib_Att', 'drib_Succ%', 'Tckl_Drib', 'Tckl_Drib%', 'PrgDist',
        'Carries_Att_3rd', 'Carries_Att_Pen', 
        'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'Touch_Live'
    ],
    'Centre-Forward': [
        'Gls', 'Ast', 'PrgR', 'Aerialwon%', 'Tack_Att_3rd', 'Tkl%', 'Tkl', 'Bolck_Pass',
        'Pass_Short', 'Pass_cmp_Short%', 'Pass_Medium', 'Pass_cmp_Medium%', 'xAG',
        'xA', 'A-xAG', 'Pass_cmp_Att_3rd', 'Touch_Att_3rd', 'Touch_Att_Pen',
        'Carries_Att_Pen', 'drib_Att', 'drib_Succ%', 
        'Sh', 'SoT', 'G/Sh', 'xG', 'npxG', 'npxG/Sh', 'G-xG'
    ]
}

In [30]:
def_pos=df.dropDuplicates(['Pos', 'Season'])
def_pos.show(100)

+------------------+---------------+---------+--------------+------+----+------+------+---+---+-------+----+----+-----+-----+----+----+-------+-----+----------+------------+------------+------------+-----+----+----+----+------+-----------+----------+----------+---+--------+---------+----------+---------------+-----------+----------------+---------+--------------+---+---+-----+----------------+----+-----+-------------+-------------+-------------+-------------+----------+--------+----------+---------+----------+-------+---------------+---------------+--------------------+-------------------+----+----+----+---+----+-------+----+------------------+------+---------+
|            Player|          Squad|   Season|   Competition|  Born|  MP|Starts|   Min|Gls|Ast|YellowC|RedC|PrgC| PrgP| PrgR| Fls| Off|Crosses|Recov|Aerialwon%|Tack_Def_3rd|Tack_Mid_3rd|Tack_Att_3rd| Tkl%| Tkl|TklW| Int|Blocks|Block_Shots|Bolck_Pass|Clearences|Err|Pass_cmp|Pass_cmp%|Pass_Short|Pass_cmp_Short%|Pass_Medium|Pass_cm

In [33]:
df = calcular_penalty_score(df)
df = calcular_performance_score(df, position_metrics)

In [6]:
df = df.toPandas()

df.to_csv("data/performnace3.csv", index=False)

df.columns

In [ ]:
df.columns

['Player',
 'Squad',
 'Season',
 'Competition',
 'Born',
 'MP',
 'Starts',
 'Min',
 'Gls',
 'Ast',
 'YellowC',
 'RedC',
 'PrgC',
 'PrgP',
 'PrgR',
 'Fls',
 'Off',
 'Crosses',
 'Recov',
 'Aerialwon%',
 'Tack_Def_3rd',
 'Tack_Mid_3rd',
 'Tack_Att_3rd',
 'Tkl%',
 'Tkl',
 'TklW',
 'Int',
 'Blocks',
 'Block_Shots',
 'Bolck_Pass',
 'Clearences',
 'Err',
 'Pass_cmp',
 'Pass_cmp%',
 'Pass_Short',
 'Pass_cmp_Short%',
 'Pass_Medium',
 'Pass_cmp_Medium%',
 'pass_Long',
 'Pass_cmp_Long%',
 'xAG',
 'xA',
 'A-xAG',
 'Pass_cmp_Att_3rd',
 'PPA',
 'CrsPA',
 'Touch_Def_3rd',
 'Touch_Mid_3rd',
 'Touch_Att_3rd',
 'Touch_Att_Pen',
 'touch_Live',
 'drib_Att',
 'drib_Succ%',
 'Tckl_Drib',
 'Tckl_Drib%',
 'PrgDist',
 'Carries_Att_3rd',
 'Carries_Att_Pen',
 'fail_To_Gain_Control',
 'Loss_Control_Tackle',
 'Sh',
 'SoT',
 'G/Sh',
 'xG',
 'npxG',
 'npxG/Sh',
 'G-xG',
 'Pos',
 'Height',
 'Value']

In [ ]:


import pandas as pd

df_fbref = pd.read_csv("data/limpios/stats/stats.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

df_fbref=df_fbref["Squad"]
df_mercado2=df_mercado["Squad"]


df_fbref.drop_duplicates().sort_values().to_csv("data/nombres_stat.csv", index=False)
df_mercado2.drop_duplicates().sort_values().to_csv("data/nombres_mercado.csv", index=False)

In [ ]:
df_mercado.columns

Index(['Player', 'Pos', 'Squad', 'Height', 'Value', 'Season', 'Competition'], dtype='object')

In [ ]:
df_mercado= df_mercado["Pos"].drop_duplicates()
df_mercado

0               Goalkeeper
4              Centre-Back
12               Left-Back
14              Right-Back
18      Defensive Midfield
21        Central Midfield
23      Attacking Midfield
26             Left Winger
29            Right Winger
30          Second Striker
32          Centre-Forward
159         Right Midfield
160          Left Midfield
4038            Midfielder
5297               Striker
Name: Pos, dtype: object

In [ ]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

In [ ]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

fbref = df_fbref["club_actual"]
understat = df_understat["club_actual"]
mercado = df_mercado["club_actual"]

nombre_fbref = fbref.dropna().sort_values().unique()
nombre_understat = understat.dropna().sort_values().unique()
nombre_mercado = mercado.dropna().sort_values().unique()

pd.Series(nombre_fbref).to_csv("prueba_data/club_actual_fbref.csv", index=False)
pd.Series(nombre_understat).to_csv("prueba_data/club_actual_understat.csv", index=False)
pd.Series(nombre_mercado).to_csv("prueba_data/club_actual_mercado.csv", index=False)

In [ ]:
duplicados = df_fbref[df_fbref.duplicated(subset=["nombre"], keep=False)]
print(duplicados)
df_ordenado = duplicados.sort_values(by="nombre")
df_ordenado.to_csv("prueba_data/duplicados_fbref.csv", index=False)

                nombre Posicion  nacido  partidos_jugados  titularidades  \
0      paxten aaronson    MF,DF  2003.0                 7              0   
1       sargis adamyan    FW,MF  1993.0                24              4   
2        karim adeyemi    FW,MF  2002.0                24             20   
3           amine adli    FW,MF  2000.0                26             16   
4            felix agu    DF,FW  1999.0                 3              0   
...                ...      ...     ...               ...            ...   
6399   piotr zielinski       MF  1994.0                26              8   
6400      nadir zortea    DF,MF  1999.0                35             33   
6401  szymon zurkowski       MF  1997.0                 5              0   
6402        milan uric       FW  1990.0                18             13   
6403        milan uric       FW  1990.0                 9              3   

      minutos_jugados  goles  asistencias  amarillas  rojas  \
0                 173   

In [ ]:
COLUMNAS_UTILES = {
    "fbref": [
        "Unnamed: 1_level_0_Player", "Unnamed: 4_level_0_Squad", "Unnamed: 3_level_0_Pos", "Unnamed: 6_level_0_Born", "Playing Time_MP", "Playing Time_Starts", "Playing Time_Min", "Performance_Gls", "Performance_Ast", "Performance_CrdY", "Performance_CrdR", "Progression_PrgC","Progression_PrgP","Progression_PrgR"
    ],
    "fbref_defense": [
       "Unnamed: 1_level_0_Player", "Unnamed: 4_level_0_Squad", "Tackles_Def 3rd", "Tackles_Mid 3rd","Tackles_Att 3rd", "Challenges_Tkl%", "Tackles_Tkl", "Tackles_TklW", "Int", "Blocks_Blocks", "Blocks_Sh", "Blocks_Pass", "Clr", "Err"
    ],
    "understat": [
        "nombre", "club_actual", "xG", "xA", "npxG", "shsots", "key_passes", "xGChain", "xGBuildup"
    ],
    "mercado": [
        "player_name", "current_club", "player_height_mtrs", "player_market_value_euro"
    ]
}

df_mercado = pd.read_csv("data/mercado/valores_mercado_LaLiga_2022-2023.csv")
mercado = ["player_name", "current_club", "player_height_mtrs", "player_market_value_euro"]
def limpiar_ligas_transfermarkt(df):
    ligas_principales = ["Serie A", "LaLiga", "Premier League", "Bundesliga"]
    return df[df["comp_name"].isin(ligas_principales)]

df_mercado = limpiar_ligas_transfermarkt(df_mercado)
columnas_utiles = COLUMNAS_UTILES.get("mercado", df_mercado.columns.tolist())
columnas_disponibles = [col for col in columnas_utiles if col in df_mercado.columns]
#df_mercado = df_mercado[mercado]
df_mercado.to_csv("prueba_data/mercadoLaLiga.csv", index=False)

In [ ]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

col = ["nombre", "club_actual", "temporada", "liga"]

fbref= df_fbref[col].sort_values(by="nombre")
defense=df_fbref_defense[col].sort_values(by="nombre")
understat=df_understat[col].sort_values(by="nombre")
mercado=df_mercado[col].sort_values(by="nombre")

fbref.to_csv("prueba_data/limpios/fbref.csv", index=False)
defense.to_csv("prueba_data/limpios/defense.csv", index=False)
understat.to_csv("prueba_data/limpios/understat.csv", index=False)
mercado.to_csv("prueba_data/limpios/mercado.csv", index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'data/limpios/fbref/fbref.csv'

In [ ]:
defense = pd.read_csv("data/possession/LaLiga_2022_2023_possession.csv")
defense.columns


Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s',
       'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen',
       'Live', 'Att', 'Succ', 'Succ%', 'Tkld', 'Tkld%', 'Carries', 'TotDist',
       'PrgDist', 'PrgC', '1/3', 'CPA', 'Mis', 'Dis', 'Rec', 'PrgR',
       'Matches'],
      dtype='object')